In [1]:
from tqdm import tqdm
 

In [2]:
from datasets import load_dataset

dataset = load_dataset("graycatHCO3/CodeAlpaca-20K-Python")


Using the latest cached version of the dataset since graycatHCO3/CodeAlpaca-20K-Python couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\XFZ\.cache\huggingface\datasets\graycatHCO3___code_alpaca-20_k-python\default\0.0.0\757f71b1aefcf442c543ac3496cdf7645b96aef5 (last modified on Tue Apr 30 16:46:40 2024).


# 1. CodeBERT

In [3]:
from transformers import AutoTokenizer

codeRoBerta_tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

def tokenize_function(examples):

    inputs = codeRoBerta_tokenizer(
        text=examples["completion"],
        text_target=examples["prompt"],
        padding="max_length", 
        truncation=True, 
        max_length=512,
        
        )

    return {
        "input_ids":inputs["input_ids"],
        "labels" : inputs["labels"]
    }

cdrba_tkd = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4777 [00:00<?, ? examples/s]

Map:   0%|          | 0/548 [00:00<?, ? examples/s]

In [6]:
from transformers import EncoderDecoderModel, RobertaConfig, RobertaModel

cdrba_encoder_config = RobertaConfig.from_pretrained("microsoft/codebert-base")
cdrba_decoder_config = RobertaConfig.from_pretrained("microsoft/codebert-base")



cdrba_decoder_config.is_decoder = True
cdrba_decoder_config.add_cross_attention = True

cdrba_decoder_config.decoder_start_token_id = 0

print("Decoder Start Token ID:", cdrba_decoder_config.decoder_start_token_id)


cdrba_encoder = RobertaModel(cdrba_encoder_config)
cdrba_decoder = RobertaModel(cdrba_decoder_config)

codeRoBerta = EncoderDecoderModel(encoder=cdrba_encoder, decoder=cdrba_decoder)


Decoder Start Token ID: 0


In [7]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=True,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=codeRoBerta,
    args=training_args,
    train_dataset=cdrba_tkd["train"],
    eval_dataset=cdrba_tkd["test"],
    tokenizer=codeRoBerta_tokenizer,
)

trainer.train()
    

  0%|          | 0/3585 [00:00<?, ?it/s]

ValueError: Make sure to set the decoder_start_token_id attribute of the model's configuration.

# 2. BART

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration